In [ ]:
import pandas as pd

In [ ]:
# Cell line structure data frame, columns are cell lines, rows are gene mutations ordered by descending frequency
mutation_df = pd.read_csv("../dataset/Cancer/binary_mutations.csv")
# Response data frame, columns have cell line name, drug name and auc score
response_df = pd.read_csv("../dataset/Cancer/cell_drug_responses.csv")
# Drug structure data frame, columns are drug name, rows are morgan footprint entries
morgan_footprints_df = pd.read_csv("../dataset/Cancer/morgan_footprints.csv")

In [ ]:
import torch
import time
from typing import Any, Union, List, Tuple, Dict
from torch.utils.data import DataLoader, TensorDataset
# import matplotlib
# matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import os
import numpy as np
from rbm import RBM
from utils import visualize_rbm, visualize_data, project_points_to_simplex, CSVDrugResponseDataset

from load_dataset import MNIST